<a href="https://colab.research.google.com/github/Kharsekina/C-homework/blob/main/AB_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats


In [23]:
from scipy.stats import ttest_1samp
import numpy as np
import pandas as pd

from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [ ]:
!pip install bootstrapped

In [25]:
df = pd.read_csv('/content/drive/MyDrive/cookie_cats.csv')

df.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


Необходимо понять пользователи какой версии дольше остаются в игре.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


Ищем нулевые значения

In [27]:
df.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

Нулевых значений нет

Проверяем кол-во уникальных айди

In [28]:
len(df['userid'].unique())

90189

Все айди уикальны, повторов нет

"Чистим" датасет. ам нужны пользователь которые минимум 1 раз вернулись в игру.

In [38]:
df_retention = df[df['retention_1'] == 1]
df_retention.head(5)


,userid,version,sum_gamerounds,retention_1,retention_7
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
8,1574,gate_40,108,True,True


Делим пользователей на 2 группы. Те, которые играли в 30 версию и в 40 версию.

In [39]:
ver_30 = df_retention[df['version']=='gate_30']
ver_30.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
1,337,gate_30,38,True,False
13,2179,gate_30,39,True,False
14,2218,gate_30,305,True,True
15,2382,gate_30,73,True,False
16,2392,gate_30,14,True,False


In [40]:
ver_40 = df_retention[df['version']=='gate_40']
ver_40.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
2,377,gate_40,165,True,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
8,1574,gate_40,108,True,True
9,1587,gate_40,153,True,False


Смотрим какая доля пользователей в каждой группе возвращалась в игру 7 раз.

In [41]:
ret_7_ver_30 = ver_30['retention_7'].mean()
ret_7_ver_30

0.3332335030448238

In [42]:
ret_7_ver_40 = ver_40['retention_7'].mean()
ret_7_ver_40

0.32337591331577115

In [43]:
z_score, p_value = sm.stats.proportions_ztest([ret_7_ver_30, ret_7_ver_40], [len(ver_30), len(ver_40)])
p_value

0.9889252044738076

p-value > alfa
Статистических различий в двух группах нет.